In [1]:
// --- Day 5: If You Give A Seed A Fertilizer ---

// Puzzle description redacted as-per Advent of Code guidelines

// You may find the puzzle description at: https://adventofcode.com/2023/day/5

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2023, 5);
WriteLines(inputLines);

Loading puzzle file: Day5.txt
Total lines: 197
Max line length: 211

seeds: 565778304 341771914 1736484943 907429186 3928647431 87620927 311881326 149873504 1588660730 119852039 1422681143 13548942 1095049712 216743334 3671387621 186617344 3055786218 213191880 2783359478 44001797

seed-to-soil map:
1136439539 28187015 34421000
4130684560 3591141854 62928737


In [4]:
record MapEntry(uint destStart, uint sourceStart, uint length) {}

class Mapper 
{
    private readonly List<MapEntry> _maps = new();

    public uint Map(uint source) {
        var matchRow = _maps.Where(m => m.sourceStart <= source && source < m.sourceStart + m.length).SingleOrDefault();
        if (matchRow == null) {
            return source;
        }

        var offset = source - matchRow.sourceStart;

        return matchRow.destStart + offset;
    }

    public void Add(uint destStart, uint sourceStart, uint length) {
        _maps.Add(new(destStart, sourceStart, length));
    }
}

var test1 = new Mapper();
test1.Add(50, 98, 2);
test1.Add(52, 50, 48);

// Seed number 79 corresponds to soil number 81.
Console.WriteLine(test1.Map(79));
// Seed number 14 corresponds to soil number 14.
Console.WriteLine(test1.Map(14));
// Seed number 55 corresponds to soil number 57.
Console.WriteLine(test1.Map(55));
// Seed number 13 corresponds to soil number 13.
Console.WriteLine(test1.Map(13));

81
14
57
13


In [5]:
var seedMapLine = inputLines[0];

Dictionary<string, Mapper> mappers = new();
Mapper currentMapper = null;
foreach (var line in inputLines.Skip(1)) 
{
    if (line.Contains("map:")) {
        currentMapper = new();
        mappers[line] = currentMapper;
    } else if (line == "") {
        currentMapper = null;
    } else {
        var numbers = line.Split(' ').Select(i => uint.Parse(i)).ToArray();
        currentMapper.Add(numbers[0], numbers[1], numbers[2]);
    }
}

string[] mapOrder = [
    "seed-to-soil map:",
    "soil-to-fertilizer map:",
    "fertilizer-to-water map:",
    "water-to-light map:",
    "light-to-temperature map:",
    "temperature-to-humidity map:",
    "humidity-to-location map:",
];

uint fullMap(uint source) 
{
    uint result = source;
    foreach (var mapper in mapOrder.Select(m => mappers[m])) {
        result = mapper.Map(result);
    }
    return result;
}

In [6]:
var seedsLine = inputLines[0];

var seeds = seedsLine.Substring(7).Split(' ').Select(s => uint.Parse((s))).ToArray();
var part1Answer = seeds.Select(fullMap).Min();

Console.WriteLine(part1Answer);

510109797


In [7]:
// 510109797 is correct!
Ensure(510109797, part1Answer);

In [8]:
// --- Part Two ---

// Puzzle description redacted as-per Advent of Code guidelines

// You may find the puzzle description at: https://adventofcode.com/2023/day/5

In [9]:
var seedPairs = Enumerable.Range(0, seeds.Length / 2).Select(i => (seeds[i], seeds[i + 1])).ToArray();
Console.WriteLine(seedPairs[0]);

(565778304, 341771914)


In [10]:
record struct Range(uint from, uint length) {}

In [11]:
class Mapper2
{
    private readonly List<MapEntry> _maps = new();

    public uint Map(uint source) {
        var matchRow = _maps.Where(m => m.sourceStart <= source && source < m.sourceStart + m.length).SingleOrDefault();
        if (matchRow == null) {
            return source;
        }

        var offset = source - matchRow.sourceStart;

        return matchRow.destStart + offset;
    }

    public void Add(uint destStart, uint sourceStart, uint length) {
        _maps.Add(new(destStart, sourceStart, length));
    }

    public IEnumerable<Range> MapRange(Range range) 
    {
        var rangeEnd = range.from + range.length;
        var rangeMatches = _maps.Where(m => m.sourceStart + m.length >= range.from && m.sourceStart < rangeEnd)
            .OrderBy(m => m.sourceStart)
            .SelectMany(m => new[] {m.sourceStart, m.sourceStart + m.length})
            .Where(r => r > range.from && r < rangeEnd)
            .Concat(new[] { range.from, rangeEnd })
            .Distinct()
            .OrderBy(r => r)
            .ToList();

        foreach (var pairs in rangeMatches.Zip(rangeMatches.Skip(1))) {
            var mapSource = Map(pairs.First);
            var r = new Range(mapSource, pairs.Second - pairs.First);
            yield return r;
        }
    }
}

var test1 = new Mapper2();
test1.Add(50, 98, 2);
test1.Add(52, 50, 48);

var testRange = new Range(3, 500);
foreach (var x in test1.MapRange(testRange)) {
    Console.WriteLine(x);
}

Range { from = 3, length = 47 }
Range { from = 52, length = 48 }
Range { from = 50, length = 2 }
Range { from = 100, length = 403 }


In [12]:
Dictionary<string, Mapper2> mappers2 = new();
{
    Mapper2 currentMapper = null;
    foreach (var line in inputLines.Skip(1)) 
    {
        if (line.Contains("map:")) {
            currentMapper = new();
            mappers2[line] = currentMapper;
        } else if (line == "") {
            currentMapper = null;
        } else {
            var numbers = line.Split(' ').Select(i => uint.Parse(i)).ToArray();
            currentMapper.Add(numbers[0], numbers[1], numbers[2]);
        }
    }
}

In [13]:
List<Range> fullMap2(Range source) 
{
    List<Range> result = [source];
    foreach (var mapper in mapOrder.Select(m => mappers2[m])) {
        result = result.SelectMany(x => mapper.MapRange(x)).ToList();
    }
    return result;
}

In [14]:
var ranges = seedPairs.Select(sp => new Range(sp.Item1, sp.Item2)).ToArray();
foreach (var res in fullMap2(ranges[0])) {
    Console.WriteLine(res);
}

Range { from = 2205270016, length = 28398111 }
Range { from = 3854006626, length = 5024044 }
Range { from = 35081694, length = 13027333 }
Range { from = 677981164, length = 6213439 }
Range { from = 722731256, length = 18180873 }
Range { from = 1068860974, length = 62085472 }
Range { from = 3660075389, length = 15038557 }
Range { from = 529753769, length = 26140330 }
Range { from = 905148721, length = 68252033 }
Range { from = 899704500, length = 5444221 }
Range { from = 2183824782, length = 14317667 }
Range { from = 2987118178, length = 17179223 }
Range { from = 4121411395, length = 21220680 }
Range { from = 2973132883, length = 13985295 }
Range { from = 2792909967, length = 9628062 }
Range { from = 2943635292, length = 1727995 }
Range { from = 2468989683, length = 4026632 }
Range { from = 2198142449, length = 5812031 }
Range { from = 2584594924, length = 6069916 }


In [15]:
// What is the lowest location number that corresponds to any of the initial seed numbers?

var part2Answer = ranges.SelectMany(range => fullMap2(range)).OrderBy(r => r.from).First().from;
Console.WriteLine(part2Answer);

9622622


In [16]:
// 9622622 is correct!
Ensure(9622622, part2Answer);